<a href="https://colab.research.google.com/github/walukas1/DeepDiveDSProjects/blob/main/Capstone_NM_Vistas_Attendance_Data_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##TO DO


1.  ✅ Build school dictionary with name and code (and eventually GPS location)
1.  ✅ Define target schools (highland cluster) and IDs
1.  ✅Organize DataFrame to have congruent format
  1. ✅ drop foster, military, migrant, FRL and Not FRL categories due to nulls
  1. ✅ drop all columns not-related to REG attendance and total populations
  1. ✅rename columns

1.  ✅ Make usable copies of DataFrames
  1.  ✅Convert data types from objects to number types (ints and floats)
  1. ✅Remove **** Masked nulls
  1. Create totals columns for 2022-2023 data set
  1. ✅rearrange to be in same order
  1. ✅ Export to CSVs on GitHub

[✅ Complete] ---
[🟡 In progress] ---
[❗ Backlog]

###Import packaged and files

In [194]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import plotly.express as px
#import pickle

# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)

In [195]:
## Path's to personal file on google drive
# SY23_24_path = '/content/drive/MyDrive/CNMDDDS/Emily/Capstone/SY 2023-24, Regular Attendance Rate, FINAL 20241106 SH Masked (1).csv'
# SY22_23_path = '/content/drive/MyDrive/CNMDDDS/Emily/Capstone/disagg_Vistas SY 2022-23, REGULAR ATTENDANCE, Feb2024 Rerun School Regular Attendance, V20240228 KF_distcolsSTATE_MASKED.csv'

## Path to file link on GitHub
SY23_24_path = 'https://raw.githubusercontent.com/DS-17-Projects-Tracking/NMPEDdata/refs/heads/main/NM2023-24Attendance.csv'
SY22_23_path = 'https://raw.githubusercontent.com/DS-17-Projects-Tracking/NMPEDdata/refs/heads/main/NM2022-23Attendance.csv'

NM22_23_df=pd.read_csv(SY22_23_path)
NM23_24_df=pd.read_csv(SY23_24_path)

 ##  ✅ Build school directory DataFrame
 with name and code (and eventually GPS location, level)

 Note: charters are labeled as their own districts. How do we categorize them

In [196]:
school_directory = {}

for i in NM23_24_df.index:
  id= int(NM23_24_df['SY2024_Accountable_Entity'].iloc[i])
  name= str(NM23_24_df['School'].iloc[i]).lower()
  district= str(NM23_24_df['District'].iloc[i]).lower()
  if name == 'statewide':
    level = 'state'
  elif name.endswith('district'):
    level = 'district'
  elif 'elementary' in name:
    level = 'es'
  elif ' middle' in name:
    level = 'ms'
  elif ' high' in name:  ### Fix this to not include school names ilike highland middle Fix: add space to front of '_high'
    level = 'hs'
  else:
    level = 'unknown level'

  if 500 <= int(NM23_24_df['SY2024_Accountable_Entity'].iloc[i]) <= 600:
    level += ' charter/special'

  # print(id,name,district, level)
  school_directory[id]= [name, district, level]

In [197]:
school_directory_df = pd.DataFrame(school_directory).transpose()
school_directory_df.columns = ('SchoolName', 'District', 'Level')
school_directory_df.index.name = 'ID'

In [198]:
school_directory_df.loc[1515:1530]

,SchoolName,District,Level
ID,,,
1515,eldorado high,albuquerque,hs
1516,nex gen academy,albuquerque,unknown level
1517,ecademy,albuquerque,unknown level
1520,highland high,albuquerque,hs
1525,la cueva high,albuquerque,hs
1530,manzano high,albuquerque,hs


In [199]:
filter = school_directory_df['SchoolName'].str.contains('')
school_directory_df[filter]['Level'].value_counts()

,count
Level,
es,400
hs,158
unknown level,150
ms,131
district,86
unknown level charter/special,42
district charter/special,13
hs charter/special,2
es charter/special,1


In [200]:
school_directory_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 984 entries, 1 to 999999
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   SchoolName  984 non-null    object
 1   District    984 non-null    object
 2   Level       984 non-null    object
dtypes: object(3)
memory usage: 63.0+ KB


In [201]:
school_directory_df.to_csv("school_directory.csv")

In [202]:
school_directory_df.reset_index().to_json("school_directory.json", orient = 'records' )

In [203]:
%%capture
%%bash
apt-get update
apt-get install -y jq

In [204]:
! jq . school_directory.json | head -20

[
  {
    "ID": 1,
    "SchoolName": "albuquerque district",
    "District": "albuquerque",
    "Level": "district"
  },
  {
    "ID": 2,
    "SchoolName": "reserve district",
    "District": "reserve",
    "Level": "district"
  },
  {
    "ID": 3,
    "SchoolName": "quemado district",
    "District": "quemado",
    "Level": "district"
  },
  {


In [205]:
# charters = []
# for i in NM23_24_df.index:
#   if 500 <= int(NM23_24_df['SY2024_Accountable_Entity'].iloc[i]) <= 600:
#     charters.append(NM23_24_df['School'].iloc[i].lower())

In [206]:
# charters

In [207]:
# school_directory_df['highland high']

##  ✅ Define target schools (highland cluster) and IDs
We need the dictionary build before we ca reference it to get IDs

In [208]:
highland_list = ['Wherry Elementary',
'Emerson Elementary',
'Whittier Elementary',
'Wilson Middle',
'Van Buren Middle',
'Hayes Middle',
'Highland High']

for i in range(0,len(highland_list)):
  highland_list[i] = highland_list[i].lower()

In [209]:
highland_list

['wherry elementary',
 'emerson elementary',
 'whittier elementary',
 'wilson middle',
 'van buren middle',
 'hayes middle',
 'highland high']

In [211]:
# highland_ids = []
# for i in highland_list:
#   highland_ids.append(int(school_directory_df[i]))  ## We need the dictionary build before we ca reference it
# # highland_list

##✅Organize DataFrames to have congruent format

### ✅ Drop sub population groups that are mostly nulls or not currently measured/relevant
- FRL
- NotFRL
- Military
- Migrant
- Foster (if applicable)

In [212]:
drop_subpop = ('FRL', 'NotFRL','Military','Migrant','Foster')
type(drop_subpop)

tuple

In [213]:
drop_cols_subpop23=[x for x in NM22_23_df.columns if x.endswith(drop_subpop)]
drop_cols_subpop23
drop_cols_subpop24=[x for x in NM23_24_df.columns if x.endswith(drop_subpop)]
drop_cols_subpop24

['AttendRate_FRL',
 'AttendRate_NotFRL',
 'AttendRate_Migrant',
 'AttendRate_Military',
 'Attendace_FRL',
 'Attendace_NotFRL',
 'Attendace_Migrant',
 'Attendace_Military',
 'Chronic_FRL',
 'Chronic_NotFRL',
 'Chronic_Migrant',
 'Chronic_Military',
 'FRL',
 'NotFRL',
 'Migrant',
 'Military',
 'Attend_rate_FRL',
 'Attend_rate_NotFRL',
 'Attend_rate_Migrant',
 'Attend_rate_Military',
 'NAttend_Reg_FRL',
 'NAttend_Reg_NotFRL',
 'NAttend_Reg_Migrant',
 'NAttend_Reg_Military',
 'NChronicRate_FRL',
 'NChronicRate_NotFRL',
 'NChronicRate_Migrant',
 'NChronicRate_Military']

In [214]:
NM22_23_df = NM22_23_df.drop(drop_cols_subpop23, axis = 1)
NM23_24_df = NM23_24_df.drop(drop_cols_subpop24, axis = 1)

NM SY 2023-2024
- Attendance = sub population's regular attendance / total population of sub pop
- Numerator  = sub population's regular attendance  [KEEP]
- Denominator = total population of sub pop [KEEP]
- 'SY2024_Accountable_Entity' = school ID [KEEP]
- 'School' = school name [KEEP]

NM SY 2022-2023
- NAttend = Reg attendance [KEEP]
- 'Name of demographic' = All [KEEP]
- SchNumb = school id [KEEP]

### ✅2023-2024 Column Uniform

Drop columns of calculated values (non-count columns)

In [215]:
NM23_24_df.columns
num_categories = 18
AttendRate = NM23_24_df.iloc[:,5:5+num_categories]
AttendACE = NM23_24_df.iloc[:,5+num_categories:5+num_categories*2]
Chronic = NM23_24_df.iloc[:,5+num_categories*2:5+num_categories*3]
totals_of_something = NM23_24_df.iloc[:,5+num_categories*3:5+num_categories*3+3]
total_count = NM23_24_df.iloc[:,8+num_categories*3:8+num_categories*4]
Attend_rate = NM23_24_df.iloc[:,8+num_categories*4:7+num_categories*5]
reg_count=NM23_24_df.iloc[:,7+num_categories*5:7+num_categories*6]
chronic_count=NM23_24_df.iloc[:,7+num_categories*6:]

# NM23_24_df.iloc[:,27+num_categories:27+num_categories + num_categories]
AttendRate           #drop
AttendACE            #drop
Chronic              #drop
totals_of_something  #drop/KEEP?
total_count          ##KEEP
Attend_rate          #drop
reg_count            ##KEEP
chronic_count         #drop/KEEP? calculate late? keep to check totals?

drop_columns23_24 = (AttendRate + AttendACE +Chronic + Attend_rate + totals_of_something).columns
drop_columns23_24

Index(['AttendRate_All', 'AttendRate_Any_Black', 'AttendRate_Any_Native',
       'AttendRate_Asian', 'AttendRate_Black', 'AttendRate_DirectCert',
       'AttendRate_EL', 'AttendRate_Female', 'AttendRate_Hispanic',
       'AttendRate_Homeless', 'AttendRate_Male', 'AttendRate_Multirace',
       'AttendRate_Native', 'AttendRate_NotDirectCert', 'AttendRate_NotEL',
       'AttendRate_NotSwD', 'AttendRate_SwD', 'AttendRate_White', 'Attend_Reg',
       'Attend_rate', 'Attend_rate_Any_Black', 'Attend_rate_Any_Native',
       'Attend_rate_Asian', 'Attend_rate_Black', 'Attend_rate_DirectCert',
       'Attend_rate_EL', 'Attend_rate_Female', 'Attend_rate_Hispanic',
       'Attend_rate_Homeless', 'Attend_rate_Male', 'Attend_rate_Multirace',
       'Attend_rate_Native', 'Attend_rate_NotDirectCert', 'Attend_rate_NotEL',
       'Attend_rate_NotSwD', 'Attend_rate_SwD', 'Attend_rate_White',
       'Attendace_All', 'Attendace_Any_Black', 'Attendace_Any_Native',
       'Attendace_Asian', 'Attendace_Black'

In [216]:
NM24 = NM23_24_df.drop(columns = drop_columns23_24).copy()

Rename columns to be uniform names by sub population (x)
- reg_X : regular attendance
- abs_X : chronically absent
- total_X : total student population

In [217]:
NAttend = [x for x in NM24.columns if x.startswith('NAttend')]
NChronic = [x for x in NM24.columns if x.startswith('NChronic')]
totals = [x for x in NM24.columns][5:23]
len(NAttend), len(NChronic), len(totals)
pd.DataFrame(list(zip(NAttend, NChronic, totals)))

reg_column_names = [f'reg_{x}' for x in totals]
abs_column_names = [f'abs_{x}' for x in totals]
totals_column_names = [f'total_{x}' for x in totals]
reg_column_names, abs_column_names
columns_renames_dict = dict(zip(NAttend + NChronic + totals, reg_column_names + abs_column_names + totals_column_names))
columns_renames_dict['SY2024_Accountable_Entity'] = 'SchoolID'
columns_renames_dict['School'] = 'SchoolName'
columns_renames_dict['Disnum']= 'DistrictID'
columns_renames_dict
# [x for x in NM24.rename(columns = columns_renames_dict)]
NM24 = NM24.rename(columns = columns_renames_dict)
NM24.columns

Index(['SchoolID', 'DistrictID', 'SchNumb', 'District', 'SchoolName',
       'total_All', 'total_Female', 'total_Male', 'total_Hispanic',
       'total_White', 'total_Black', 'total_Asian', 'total_Native',
       'total_Multirace', 'total_DirectCert', 'total_NotDirectCert',
       'total_SwD', 'total_NotSwD', 'total_EL', 'total_NotEL',
       'total_Homeless', 'total_Any_Native', 'total_Any_Black', 'reg_All',
       'reg_Female', 'reg_Male', 'reg_Hispanic', 'reg_White', 'reg_Black',
       'reg_Asian', 'reg_Native', 'reg_Multirace', 'reg_DirectCert',
       'reg_NotDirectCert', 'reg_SwD', 'reg_NotSwD', 'reg_EL', 'reg_NotEL',
       'reg_Homeless', 'reg_Any_Native', 'reg_Any_Black', 'abs_All',
       'abs_Female', 'abs_Male', 'abs_Hispanic', 'abs_White', 'abs_Black',
       'abs_Asian', 'abs_Native', 'abs_Multirace', 'abs_DirectCert',
       'abs_NotDirectCert', 'abs_SwD', 'abs_NotSwD', 'abs_EL', 'abs_NotEL',
       'abs_Homeless', 'abs_Any_Native', 'abs_Any_Black'],
      dtype='object

### ✅2022-2023 Column Uniform

Drop columns of calculated values (non-count columns)

In [218]:
NM23  = NM22_23_df.copy()
drop_columns22_23= [x for x in NM22_23_df.columns if x.startswith('Attendance')]
NM23  = NM22_23_df.copy().drop(columns = drop_columns22_23)

numerators = [x for x in NM23.columns if x.startswith('Num')]
denominators =  [x for x in NM23.columns if x.startswith('Denom')]
list(zip(numerators, denominators, totals))

[('Numerator.Attendance.1.All', 'Denominator.Attendance.1.All', 'All'),
 ('Numerator.Attendance.2.Female',
  'Denominator.Attendance.2.Female',
  'Female'),
 ('Numerator.Attendance.3.Male', 'Denominator.Attendance.3.Male', 'Male'),
 ('Numerator.Attendance.4.Hispanic',
  'Denominator.Attendance.4.Hispanic',
  'Hispanic'),
 ('Numerator.Attendance.5.White', 'Denominator.Attendance.5.White', 'White'),
 ('Numerator.Attendance.6.Black', 'Denominator.Attendance.6.Black', 'Black'),
 ('Numerator.Attendance.7.Asian', 'Denominator.Attendance.7.Asian', 'Asian'),
 ('Numerator.Attendance.8.Native',
  'Denominator.Attendance.8.Native',
  'Native'),
 ('Numerator.Attendance.9.Multirace',
  'Denominator.Attendance.9.Multirace',
  'Multirace'),
 ('Numerator.Attendance.12.DirectCert',
  'Denominator.Attendance.12.DirectCert',
  'DirectCert'),
 ('Numerator.Attendance.13.NotDirectCert',
  'Denominator.Attendance.13.NotDirectCert',
  'NotDirectCert'),
 ('Numerator.Attendance.14.SwD', 'Denominator.Attendance.

Rename columns to be uniform names by sub population (x)
- reg_X : regular attendance
- abs_X : chronically absent
- total_X : total student population

In [219]:
columns_renames_dict = columns_renames_dict | dict(zip(numerators + denominators, reg_column_names + abs_column_names))
columns_renames_dict['SchNumb'] = 'SchoolID'
columns_renames_dict['DistrictNumber'] = 'DistrictID'

NM23 = NM23.rename(columns = columns_renames_dict)

In [220]:
NM23.columns

Index(['DistrictID', 'SchoolID', 'reg_All', 'reg_Female', 'reg_Male',
       'reg_Hispanic', 'reg_White', 'reg_Black', 'reg_Asian', 'reg_Native',
       'reg_Multirace', 'reg_DirectCert', 'reg_NotDirectCert', 'reg_SwD',
       'reg_NotSwD', 'reg_EL', 'reg_NotEL', 'reg_Homeless', 'reg_Any_Native',
       'reg_Any_Black', 'abs_All', 'abs_Female', 'abs_Male', 'abs_Hispanic',
       'abs_White', 'abs_Black', 'abs_Asian', 'abs_Native', 'abs_Multirace',
       'abs_DirectCert', 'abs_NotDirectCert', 'abs_SwD', 'abs_NotSwD',
       'abs_EL', 'abs_NotEL', 'abs_Homeless', 'abs_Any_Native',
       'abs_Any_Black'],
      dtype='object')

In [221]:
column_order = [x for x in NM23.columns]
NM23.shape, NM24.shape

((1000, 38), (984, 59))

In [222]:
NM24 = NM24[column_order+ totals_column_names]
NM23 = NM23[column_order]
NM23.shape, NM24.shape
list(zip(NM23.columns, NM24.columns))

[('DistrictID', 'DistrictID'),
 ('SchoolID', 'SchoolID'),
 ('reg_All', 'reg_All'),
 ('reg_Female', 'reg_Female'),
 ('reg_Male', 'reg_Male'),
 ('reg_Hispanic', 'reg_Hispanic'),
 ('reg_White', 'reg_White'),
 ('reg_Black', 'reg_Black'),
 ('reg_Asian', 'reg_Asian'),
 ('reg_Native', 'reg_Native'),
 ('reg_Multirace', 'reg_Multirace'),
 ('reg_DirectCert', 'reg_DirectCert'),
 ('reg_NotDirectCert', 'reg_NotDirectCert'),
 ('reg_SwD', 'reg_SwD'),
 ('reg_NotSwD', 'reg_NotSwD'),
 ('reg_EL', 'reg_EL'),
 ('reg_NotEL', 'reg_NotEL'),
 ('reg_Homeless', 'reg_Homeless'),
 ('reg_Any_Native', 'reg_Any_Native'),
 ('reg_Any_Black', 'reg_Any_Black'),
 ('abs_All', 'abs_All'),
 ('abs_Female', 'abs_Female'),
 ('abs_Male', 'abs_Male'),
 ('abs_Hispanic', 'abs_Hispanic'),
 ('abs_White', 'abs_White'),
 ('abs_Black', 'abs_Black'),
 ('abs_Asian', 'abs_Asian'),
 ('abs_Native', 'abs_Native'),
 ('abs_Multirace', 'abs_Multirace'),
 ('abs_DirectCert', 'abs_DirectCert'),
 ('abs_NotDirectCert', 'abs_NotDirectCert'),
 ('abs_Sw

## ✅ Make DataSet usable


### ✅ Convert Data Types
Do this after all rearranging is complete!

- numeric values to floats
- blanks to zeros   (!!! Ask Steve if this is a correct assumption)
- ***** to nulls

In [235]:
orig_NM24 = NM24.copy()
orig_NM23 = NM23.copy()

In [236]:
coerced_df23 = pd.DataFrame()
for i in NM23.columns:
  coerced_df23[i] = pd.to_numeric(NM23[i].replace(r'(| +)', '0', regex = True), errors='coerce')

coerced_df23.shape, NM23.shape

((1000, 38), (1000, 38))

In [237]:
coerced_df24 = pd.DataFrame()
for i in NM24.columns:
  coerced_df24[i] = pd.to_numeric(NM24[i].replace(r'(| +)', '0', regex = True), errors='coerce')

coerced_df24.shape, NM24.shape

((984, 56), (984, 56))

### ✅ Create Totals columns for 2022-2023 dataset
(Now that we can actually work with the numbers)

In [238]:
to_total_reg = coerced_df23[reg_column_names]
to_total_abs = coerced_df23[abs_column_names]

to_total_reg.columns = [None] * len(to_total_reg.columns)
to_total_abs.columns = [None] * len(to_total_abs.columns)

totals23 = to_total_reg + to_total_abs
totals23.columns = totals_column_names

In [239]:
# ### Check to see if totals added correctly
pd.concat([coerced_df23[abs_column_names].iloc[4].reset_index() ,
coerced_df23[reg_column_names].iloc[4].reset_index() ,
totals23.iloc[4].reset_index()
], axis = 1) #.iloc[:, 1::2].apply(lambda x : x.iloc[0]+x.iloc[1] == x.iloc[2], axis = 1)

,index,4,index,4,index,4
0,abs_All,307060.0,reg_All,304030.0,total_All,611090.0
1,abs_Female,108070.0,reg_Female,107050.0,total_Female,215120.0
2,abs_Male,108090.0,reg_Male,106080.0,total_Male,214170.0
3,abs_Hispanic,100020.0,reg_Hispanic,9040.0,total_Hispanic,109060.0
4,abs_White,304010.0,reg_White,300090.0,total_White,604100.0
5,abs_Black,NaN,reg_Black,90.0,total_Black,NaN
6,abs_Asian,NaN,reg_Asian,1000.0,total_Asian,NaN
7,abs_Native,1030.0,reg_Native,1030.0,total_Native,2060.0
8,abs_Multirace,NaN,reg_Multirace,NaN,total_Multirace,NaN
9,abs_DirectCert,5080.0,reg_DirectCert,4070.0,total_DirectCert,9150.0


Add year column inoder to combine data sets

In [240]:
coerced_df24['Year'] = 2024
coerced_df23['Year'] = 2023

In [241]:
coerced_df24['Year'].value_counts()
coerced_df23['Year'].value_counts()

,count
Year,
2023,1000


In [242]:
coerced_df23_w_totals = pd.concat([coerced_df23, totals23], axis = 1)
coerced_df23_w_totals.head()

,DistrictID,SchoolID,reg_All,reg_Female,reg_Male,reg_Hispanic,reg_White,reg_Black,reg_Asian,reg_Native,...,total_Multirace,total_DirectCert,total_NotDirectCert,total_SwD,total_NotSwD,total_EL,total_NotEL,total_Homeless,total_Any_Native,total_Any_Black
0,1,1000,5060.0,2040.0,3020.0,3030.0,4090.0,NaN,0.0,NaN,...,NaN,3020.0,9120.0,12140.0,0.0,NaN,10170.0,NaN,NaN,NaN
1,1,1006,207030.0,104070.0,102060.0,4000.0,NaN,NaN,0.0,203070.0,...,6140.0,303110.0,309140.0,106160.0,506090.0,105040.0,509010.0,NaN,611050.0,3030.0
2,1,1016,7060.0,4030.0,3030.0,6040.0,6000.0,NaN,NaN,NaN,...,NaN,104140.0,7120.0,3080.0,109080.0,6060.0,106100.0,NaN,1080.0,NaN
3,1,1017,4020.0,2010.0,2010.0,3050.0,3050.0,NaN,NaN,NaN,...,NaN,105130.0,NaN,5020.0,11090.0,NaN,104090.0,NaN,NaN,NaN
4,1,1024,304030.0,107050.0,106080.0,9040.0,300090.0,90.0,1000.0,1030.0,...,NaN,9150.0,510140.0,6080.0,604110.0,NaN,611070.0,0.0,2100.0,NaN


### ✅Rearrange order of columns to match

In [243]:
column_order = ['Year','DistrictID', 'SchoolID', 'reg_All', 'reg_Female', 'reg_Male',
       'reg_Hispanic', 'reg_White', 'reg_Black', 'reg_Asian', 'reg_Native',
       'reg_Multirace', 'reg_DirectCert', 'reg_NotDirectCert', 'reg_SwD',
       'reg_NotSwD', 'reg_EL', 'reg_NotEL', 'reg_Homeless', 'reg_Any_Native',
       'reg_Any_Black', 'abs_All', 'abs_Female', 'abs_Male', 'abs_Hispanic',
       'abs_White', 'abs_Black', 'abs_Asian', 'abs_Native', 'abs_Multirace',
       'abs_DirectCert', 'abs_NotDirectCert', 'abs_SwD', 'abs_NotSwD',
       'abs_EL', 'abs_NotEL', 'abs_Homeless', 'abs_Any_Native',
       'abs_Any_Black', 'total_All', 'total_Female', 'total_Male',
       'total_Hispanic', 'total_White', 'total_Black', 'total_Asian',
       'total_Native', 'total_Multirace', 'total_DirectCert',
       'total_NotDirectCert', 'total_SwD', 'total_NotSwD', 'total_EL',
       'total_NotEL', 'total_Homeless', 'total_Any_Native', 'total_Any_Black']
# column_order
pd.DataFrame(list(zip(coerced_df23_w_totals[column_order].columns, coerced_df24[column_order].columns)))

,0,1
0,Year,Year
1,DistrictID,DistrictID
2,SchoolID,SchoolID
3,reg_All,reg_All
4,reg_Female,reg_Female
5,reg_Male,reg_Male
6,reg_Hispanic,reg_Hispanic
7,reg_White,reg_White
8,reg_Black,reg_Black
9,reg_Asian,reg_Asian


In [244]:
attendance_NMPED_df22_23_23_24 = pd.concat([coerced_df23_w_totals[column_order], coerced_df24[column_order]], axis=0, ignore_index=True)

In [245]:
attendance_NMPED_df22_23_23_24.dtypes

,0
Year,int64
DistrictID,float64
SchoolID,int64
reg_All,float64
reg_Female,float64
reg_Male,float64
reg_Hispanic,float64
reg_White,float64
reg_Black,float64
reg_Asian,float64


### ✅ Export coerced dataframes and originals to GitHub

In [247]:
attendance_NMPED_df22_23_23_24.to_json("attendance_NMPED_df22_23_23_24.json", orient = 'records')